# Machine Learning pour prédire une valeur foncière

Dans ce notebook, vous trouvez notre code d'entrainement et validation automatique d'algorithmes de Machine Learning pour prédire une valeur foncière à partir du jeu de données *Demandes de valeurs foncières* du premier trimestre de 2020.

**dataset:** https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/<br>
**tâche:** Régression<br>
**target variable:** *Valeur fonciere*

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.impute import SimpleImputer
#pour l'entraînement automatique:
from supervised.automl import AutoML
#pour générer un rapport html des résultats:
import IPython
import markdown

## 1. Importation des données

On importe le dataset à partir de son url.

In [16]:
url_2020 = "https://www.data.gouv.fr/fr/datasets/r/90a98de0-f562-4328-aa16-fe0dd1dca60f"
data_2020 = pd.read_csv(url_2020, sep='|')  #dataframe des demandes de valeurs foncières du premier trimestre de 2020

Columns (14,18,23,24,26,28,31,33,41) have mixed types.Specify dtype option on import or set low_memory=False.


## 2. Preprocessing

### 2.1. Elimination des colonnes non supportées d'après Pandas Profiling

In [17]:
data_2020 = data_2020.drop(columns=['1er lot','Code departement','Code service CH', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI','No Volume','2eme lot','3eme lot','Identifiant local'])

### 2.2. Modification du type des variables

In [18]:
#target variable
data_2020['Valeur fonciere'] = data_2020['Valeur fonciere'].str.replace(',', '.', regex=True)
data_2020['Valeur fonciere'] = data_2020['Valeur fonciere'].astype(float)

#variables catégorielles
categorical_features = ['Nature culture', 'Type local','Code type local', 'Nature mutation']

for c in categorical_features:
    data_2020[c] = data_2020[c].astype('category').cat.codes

### 2.3. Imputation des valeurs manquantes

Pour pouvoir entraîner nos modèles, on élimine les lignes du datasets correspondant à des valeurs manquantes de la variable réponse. On élimine également les variables avec plus de 90% de valeurs manquantes.

In [19]:
data_2020 = data_2020.drop(columns=[col for col in data_2020.columns if data_2020[col].isna().sum()/len(data_2020)>0.9])
data_2020 = data_2020[data_2020['Valeur fonciere'].isna()==False]

In [22]:
data_2020.columns

Index(['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code commune', 'Section', 'No plan', 'Nombre de lots',
       'Code type local', 'Type local', 'Surface reelle bati',
       'Nombre pieces principales', 'Nature culture', 'Surface terrain'],
      dtype='object')

### 2.3. Séléction de variables

In [23]:
features = ['Nature mutation', 'Valeur fonciere', 'Type de voie','Section', 'Nombre de lots',
       'Code type local', 'Type local', 'Surface reelle bati',
       'Nombre pieces principales', 'Nature culture',
       'Surface terrain']

In [24]:
data = data_2020[features]

## 3. Train/test splitting

On définit la variable réponse et les variables explicatives.<br>
On partage le dataset en dataset d'entraînement et de test. Attention, AutoML nécessite d'un partage 75%/25%.

In [25]:
y = data['Valeur fonciere'].values
X = data.drop(columns=['Valeur fonciere'])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## 4.AutoML

On entraîne, test et valide automatiquement plusieurs algorithmes de ML sur le dataset avec AutoML (plus d'infos ici: https://supervised.mljar.com/) et on génère un html des métriques et des paramètres du modèle.

In [28]:
automl = AutoML(total_time_limit=7*60,mode='Perform',random_state=42)
#fit model
automl.fit(X_train,y_train)
#predictions
predictions = automl.predict(X_test)
#generate html report
automl.report()